In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
path = "data/cleaned_data.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
data = df.copy()

In [5]:
data.shape

(15215, 35)

In [6]:
data.columns

Index(['permit number', 'permit type', 'permit type definition',
       'permit creation date', 'block', 'lot', 'street number', 'street name',
       'street suffix', 'description', 'current status', 'current status date',
       'filed date', 'issued date', 'completed date',
       'first construction document date', 'number of existing stories',
       'number of proposed stories', 'permit expiration date',
       'estimated cost', 'revised cost', 'existing use', 'existing units',
       'proposed use', 'proposed units', 'plansets',
       'existing construction type', 'existing construction type description',
       'proposed construction type', 'proposed construction type description',
       'supervisor district', 'neighborhoods - analysis boundaries', 'zipcode',
       'location', 'record id'],
      dtype='object')

In [7]:
data.head()

,permit number,permit type,permit type definition,permit creation date,block,lot,street number,street name,street suffix,description,current status,current status date,filed date,issued date,completed date,first construction document date,number of existing stories,number of proposed stories,permit expiration date,estimated cost,revised cost,existing use,existing units,proposed use,proposed units,plansets,existing construction type,existing construction type description,proposed construction type,proposed construction type description,supervisor district,neighborhoods - analysis boundaries,zipcode,location,record id
0,201505000000.0,4,sign - erect,2015-05-06,326,23,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,2015-05-06,2015-11-09,2013-03-21,2015-11-09,6.0,6.0,2016-11-03,4000.0,4000.0,tourist hotel/motel,143.0,museum,160.0,2.0,3.0,constr type 3,1.0,constr type 1,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1.380610e+12
1,201604000000.0,4,sign - erect,2016-04-19,306,7,440,Geary,St,remove (e) awning and associated signs.,issued,08/03/2017,2016-04-19,2017-08-03,2013-03-21,2017-08-03,7.0,7.0,2017-12-03,1.0,500.0,tourist hotel/motel,36.0,misc group residns.,36.0,2.0,3.0,constr type 3,2.0,constr type 2,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1.420160e+12
2,201605000000.0,3,additions alterations or repairs,2016-05-27,595,203,1647,Pacific,Av,installation of separating wall,withdrawn,09/26/2017,2016-05-27,2013-06-03,2013-03-21,2013-06-03,6.0,6.0,2014-05-29,20000.0,25000.0,retail sales,39.0,retail sales,39.0,2.0,1.0,constr type 1,1.0,constr type 1,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1.424860e+12
3,201611000000.0,8,otc alterations permit,2016-11-07,156,11,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,complete,07/24/2017,2016-11-07,2017-07-18,2017-07-24,2017-07-18,2.0,2.0,2018-07-13,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1.443570e+12
4,201611000000.0,6,demolitions,2016-11-28,342,1,950,Market,St,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,2016-11-28,2017-12-01,2013-03-21,2017-11-20,3.0,3.0,2018-12-01,100000.0,100000.0,retail sales,5.0,workshop commercial,8.0,2.0,3.0,constr type 3,1.0,constr type 1,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",1.445480e+11


In [8]:
data.select_dtypes(include=['float', 'int']).corr()

,permit type,street number,number of existing stories,number of proposed stories,estimated cost,revised cost,existing units,proposed units,plansets,existing construction type,proposed construction type,supervisor district,zipcode,record id
permit type,1.000000,-0.016908,0.022323,0.018089,-0.108060,-0.107840,-0.007071,-0.016602,-0.245464,0.067909,0.067179,-0.000477,0.014770,0.014865
street number,-0.016908,1.000000,-0.175092,-0.175230,-0.017012,-0.017333,-0.051983,-0.054152,-0.039319,0.163335,0.168474,-0.126037,0.114524,-0.057987
number of existing stories,0.022323,-0.175092,1.000000,0.998669,0.081559,0.067786,0.322734,0.332505,0.208006,-0.558950,-0.577779,-0.133345,-0.277345,-0.094342
number of proposed stories,0.018089,-0.175230,0.998669,1.000000,0.092160,0.076694,0.322606,0.333113,0.210115,-0.558399,-0.579647,-0.131927,-0.276871,-0.093729
estimated cost,-0.108060,-0.017012,0.081559,0.092160,1.000000,0.869941,0.029702,0.036769,0.032138,-0.036240,-0.049904,0.005449,-0.020189,0.006076
revised cost,-0.107840,-0.017333,0.067786,0.076694,0.869941,1.000000,0.025463,0.034102,0.030057,-0.030625,-0.047600,0.010443,-0.013260,0.007679
existing units,-0.007071,-0.051983,0.322734,0.322606,0.029702,0.025463,1.000000,0.990318,0.024635,-0.224122,-0.233958,-0.050783,-0.109920,-0.010938
proposed units,-0.016602,-0.054152,0.332505,0.333113,0.036769,0.034102,0.990318,1.000000,0.028369,-0.229500,-0.242780,-0.048645,-0.114787,-0.008779
plansets,-0.245464,-0.039319,0.208006,0.210115,0.032138,0.030057,0.024635,0.028369,1.000000,-0.331010,-0.317295,-0.046011,-0.135674,-0.042171
existing construction type,0.067909,0.163335,-0.558950,-0.558399,-0.036240,-0.030625,-0.224122,-0.229500,-0.331010,1.000000,0.926753,0.137015,0.309988,0.078448


In [9]:
data["permit creation date"] = pd.to_datetime(data["permit creation date"])
data["filed date"] = pd.to_datetime(data["filed date"])
data["first construction document date"] = pd.to_datetime(data["first construction document date"])
data["issued date"] = pd.to_datetime(data["issued date"])
data["permit expiration date"] = pd.to_datetime(data["permit expiration date"])
data["permit creation date"] = pd.to_datetime(data["permit creation date"])
data["current status date"] = pd.to_datetime(data["current status date"])
data["completed date"] = pd.to_datetime(data["completed date"])


In [10]:
data["if_permit_created_on_file_date"] = [data["filed date"] == data["permit creation date"]]
data["if_permit_created_on_file_date"].value_counts()

True     15193
False       22
Name: if_permit_created_on_file_date, dtype: int64

In [11]:
# if current_status is complete
data["if_completed"] = data["current status"] == 'complete'
data["if_completed"].value_counts()

True     9026
False    6189
Name: if_completed, dtype: int64

In [12]:
# if first construction document date on issue date
data["if_first_construc_is_issue_date"] = data["first construction document date"] == data["issued date"]
data["if_first_construc_is_issue_date"].value_counts()

True     14957
False      258
Name: if_first_construc_is_issue_date, dtype: int64

In [13]:
data["diff_permit_date"] = data["permit expiration date"] - data["permit creation date"]
data["diff_permit_date"].value_counts()

360 days      6874
361 days       637
363 days       279
364 days       244
365 days       219
362 days       208
366 days       180
367 days       166
373 days       123
374 days       123
1080 days      107
368 days       100
370 days        91
372 days        82
371 days        75
381 days        74
378 days        73
398 days        68
375 days        67
377 days        62
380 days        59
369 days        55
379 days        54
1081 days       53
388 days        52
391 days        52
401 days        51
399 days        51
408 days        50
402 days        47
394 days        47
400 days        47
406 days        45
497 days        43
353 days        43
387 days        41
405 days        41
386 days        41
427 days        41
471 days        41
422 days        41
437 days        40
416 days        40
507 days        40
458 days        39
385 days        39
384 days        39
393 days        39
395 days        39
382 days        38
492 days        38
357 days        38
392 days    

In [14]:
data["expired_before_completed"] = data["completed date"] > data["permit expiration date"]
data["expired_before_completed"].value_counts()

False    14243
True       972
Name: expired_before_completed, dtype: int64

In [15]:
data["ratio_stories"] = data["number of existing stories"] / data["number of proposed stories"]
data["ratio_stories"].value_counts()

1.000000    14393
0.666667      182
0.750000       82
0.500000       69
0.000000       47
1.090909       29
1.083333       27
1.200000       25
1.100000       24
1.111111       24
0.333333       19
2.000000       18
0.400000       15
1.333333       15
1.076923       12
1.250000       10
1.062500        9
1.125000        9
0.950000        9
0.833333        8
inf             7
0.971429        6
0.800000        6
0.967742        6
1.166667        6
1.045455        6
3.000000        6
1.066667        5
0.941176        5
0.250000        5
1.181818        5
0.947368        5
1.142857        5
1.071429        4
1.041667        4
0.972222        4
1.033333        4
0.900000        3
0.875000        3
0.888889        3
1.272727        3
0.968750        3
0.970588        3
1.047619        3
0.966667        3
0.923077        3
1.032258        3
0.954545        2
0.962963        2
0.857143        2
1.043478        2
1.040000        2
0.600000        2
1.500000        2
1.133333        2
0.882353  

In [16]:
data["existing_proposed_construction_type"] = data["existing construction type"] == data["proposed construction type"]
data["existing_proposed_construction_type"].value_counts()

True     13559
False     1656
Name: existing_proposed_construction_type, dtype: int64

In [17]:
data["ratio_estimated_revised"] = data["estimated cost"] / data["revised cost"]
data["ratio_estimated_revised"].value_counts()

1.000000e+00    9847
inf              346
3.000000e+03     154
5.000000e-01     146
4.000000e+03     124
6.666667e-01     123
1.000000e+03     120
5.000000e+03     116
2.000000e+03     110
1.000000e+04     104
8.333333e-01      93
8.000000e-01      83
6.000000e-01      71
6.000000e+03      70
5.000000e+02      64
4.000000e-01      61
1.500000e+04      55
7.500000e-01      55
8.000000e+03      53
3.333333e-01      52
6.250000e-01      49
1.000000e+02      49
2.500000e+03      46
3.500000e+03      44
1.500000e+03      43
6.500000e+03      41
2.800000e+03      40
2.000000e-01      36
3.700000e+03      35
7.142857e-01      33
5.555556e-01      32
1.200000e+04      32
2.000000e+02      32
4.500000e+03      31
2.500000e-01      29
1.000000e+05      29
9.000000e+03      28
3.750000e-01      26
9.000000e-01      25
3.741955e-03      24
1.800000e+03      23
5.714286e-01      23
1.600000e+03      22
8.888889e-01      22
1.000000e-01      21
4.166667e-01      21
1.400000e+04      21
4.200000e+03 

In [18]:
data["ratio_existing_proposed_units"] = data["existing units"] / data["proposed units"]
data["ratio_existing_proposed_units"].value_counts()

1.000000     9483
0.000000      414
inf           253
0.500000       96
0.666667       90
2.000000       53
0.750000       51
1.500000       38
1.250000       36
1.666667       30
0.833333       29
1.333333       28
1.166667       28
0.333333       26
0.857143       21
1.125000       20
1.111111       20
1.200000       19
0.800000       19
0.600000       17
0.875000       16
0.250000       16
0.933333       13
0.888889       11
0.909091       10
0.952381       10
1.090909        9
0.900000        8
1.050000        7
1.071429        7
0.923077        7
1.800000        7
0.944444        7
1.060606        7
1.047619        7
0.954545        7
0.400000        6
0.937500        6
0.916667        6
1.083333        6
0.625000        6
0.777778        5
0.960000        5
3.000000        5
0.555556        5
1.100000        5
0.714286        5
1.055556        5
1.013889        5
0.969697        5
0.818182        5
2.500000        4
0.928571        4
0.964286        4
1.058824        4
1.040000  

In [19]:
mm = MinMaxScaler()
data["street number"] = mm.fit_transform(data[["street number"]])

In [20]:
# extracting longitude and latitude from location

data["latitude"] = data['location'].apply(lambda row: float(row.strip("()").split(", ")[0]))
data["longitude"] = data['location'].apply(lambda row: float(row.strip("()").split(", ")[1]))

In [21]:
# columns to drop
col_drop = ["location", "record id", "description", "permit number", 
            "existing construction type description", "proposed construction type description", 
            "street name", "lot", "block", "permit type definition"]

In [22]:
data = data.drop(columns=col_drop) # drop columns

In [23]:
data.head()

,permit type,permit creation date,street number,street suffix,current status,current status date,filed date,issued date,completed date,first construction document date,number of existing stories,number of proposed stories,permit expiration date,estimated cost,revised cost,existing use,existing units,proposed use,proposed units,plansets,existing construction type,proposed construction type,supervisor district,neighborhoods - analysis boundaries,zipcode,if_permit_created_on_file_date,if_completed,if_first_construc_is_issue_date,diff_permit_date,expired_before_completed,ratio_stories,existing_proposed_construction_type,ratio_estimated_revised,ratio_existing_proposed_units,latitude,longitude
0,4,2015-05-06,0.016953,St,expired,2017-12-21,2015-05-06,2015-11-09,2013-03-21,2015-11-09,6.0,6.0,2016-11-03,4000.0,4000.0,tourist hotel/motel,143.0,museum,160.0,2.0,3.0,1.0,3.0,Tenderloin,94102.0,True,False,True,547 days,False,1.0,False,1.000,0.89375,37.785719,-122.408523
1,4,2016-04-19,0.053543,St,issued,2017-08-03,2016-04-19,2017-08-03,2013-03-21,2017-08-03,7.0,7.0,2017-12-03,1.0,500.0,tourist hotel/motel,36.0,misc group residns.,36.0,2.0,3.0,2.0,3.0,Tenderloin,94102.0,True,False,True,593 days,False,1.0,False,0.002,1.00000,37.787340,-122.410632
2,3,2016-05-27,0.200756,Av,withdrawn,2017-09-26,2016-05-27,2013-06-03,2013-03-21,2013-06-03,6.0,6.0,2014-05-29,20000.0,25000.0,retail sales,39.0,retail sales,39.0,2.0,1.0,1.0,3.0,Russian Hill,94109.0,True,False,True,-729 days,False,1.0,True,0.800,1.00000,37.794657,-122.422326
3,8,2016-11-07,0.149896,Av,complete,2017-07-24,2016-11-07,2017-07-18,2017-07-24,2017-07-18,2.0,2.0,2018-07-13,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,5.0,5.0,3.0,Nob Hill,94109.0,True,True,True,613 days,False,1.0,True,1.000,1.00000,37.795959,-122.415574
4,6,2016-11-28,0.115746,St,issued,2017-12-01,2016-11-28,2017-12-01,2013-03-21,2017-11-20,3.0,3.0,2018-12-01,100000.0,100000.0,retail sales,5.0,workshop commercial,8.0,2.0,3.0,1.0,6.0,Tenderloin,94102.0,True,False,False,733 days,False,1.0,False,1.000,0.62500,37.783153,-122.409509


In [24]:
# using labelEncoder to encode object datatypes
obj_cols = data.select_dtypes(include="object").columns.to_list()
obj_cols

['street suffix',
 'current status',
 'existing use',
 'proposed use',
 'neighborhoods - analysis boundaries']

In [25]:
le = LabelEncoder()
le.fit_transform(data["street suffix"])

array([12, 12,  1, ..., 12, 12, 12])

In [26]:
#using a for loop encode obj_col with labelencoder
for col in obj_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [27]:
data[obj_cols].head()

,street suffix,current status,existing use,proposed use,neighborhoods - analysis boundaries
0,12,4,56,39,34
1,12,6,56,34,34
2,1,11,47,54,30
3,1,3,0,0,19
4,12,6,47,67,34


In [28]:
# encode the datetime variables using toordinal()

datetime_cols = data.select_dtypes("datetime").columns.to_list()
datetime_cols

['permit creation date',
 'current status date',
 'filed date',
 'issued date',
 'completed date',
 'first construction document date',
 'permit expiration date']

In [31]:
for date in datetime_cols:
    data[f"{date}_col"] = data[date].apply(lambda row: row.toordinal())

In [32]:
data = data.drop(columns=datetime_cols)

In [33]:
data.head()

,permit type,street number,street suffix,current status,number of existing stories,number of proposed stories,estimated cost,revised cost,existing use,existing units,proposed use,proposed units,plansets,existing construction type,proposed construction type,supervisor district,neighborhoods - analysis boundaries,zipcode,if_permit_created_on_file_date,if_completed,if_first_construc_is_issue_date,diff_permit_date,expired_before_completed,ratio_stories,existing_proposed_construction_type,ratio_estimated_revised,ratio_existing_proposed_units,latitude,longitude,permit creation date_col,current status date_col,filed date_col,issued date_col,completed date_col,first construction document date_col,permit expiration date_col
0,4,0.016953,12,4,6.0,6.0,4000.0,4000.0,56,143.0,39,160.0,2.0,3.0,1.0,3.0,34,94102.0,True,False,True,547 days,False,1.0,False,1.000,0.89375,37.785719,-122.408523,735724,736684,735724,735911,734948,735911,736271
1,4,0.053543,12,6,7.0,7.0,1.0,500.0,56,36.0,34,36.0,2.0,3.0,2.0,3.0,34,94102.0,True,False,True,593 days,False,1.0,False,0.002,1.00000,37.787340,-122.410632,736073,736544,736073,736544,734948,736544,736666
2,3,0.200756,1,11,6.0,6.0,20000.0,25000.0,47,39.0,54,39.0,2.0,1.0,1.0,3.0,30,94109.0,True,False,True,-729 days,False,1.0,True,0.800,1.00000,37.794657,-122.422326,736111,736598,736111,735022,734948,735022,735382
3,8,0.149896,1,3,2.0,2.0,2000.0,2000.0,0,1.0,0,1.0,2.0,5.0,5.0,3.0,19,94109.0,True,True,True,613 days,False,1.0,True,1.000,1.00000,37.795959,-122.415574,736275,736534,736275,736528,736534,736528,736888
4,6,0.115746,12,6,3.0,3.0,100000.0,100000.0,47,5.0,67,8.0,2.0,3.0,1.0,6.0,34,94102.0,True,False,False,733 days,False,1.0,False,1.000,0.62500,37.783153,-122.409509,736296,736664,736296,736664,734948,736653,737029


In [35]:
# TODO use elasticnet to select important features
# TODO check distribution, if not normal convert it to normal using log transformation
# TODO add comments

In [37]:
data.to_csv("data/feature_engineering.csv", index=False) #export to csv